# Homework

Before you begin, remember to import the necessary libraries.

In [19]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
sns.set_theme()

import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


We will load the `kc_housing_data.csv` dataset from the `data` folder to a new DataFrame named `df`. The data set contains data relative to *home sales prices and characteristics for Seattle and King County, WA between May 2014 and 2015* ([source](https://geodacenter.github.io/data-and-lab/KingCounty-HouseSales2015/)). 

In [20]:
df = pd.read_csv("data/kc_housing_data.csv")

Always familiarize a little with the data using .head() and .info() 

In [21]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

Now let's design and fit a multiple linear regression model, using `price` as the dependent variable and the following predictors `['bedrooms', 'bathrooms', 'view', 'grade']`. 

In [23]:
Y = df['price']
X = df[['bedrooms','bathrooms','view','grade']]
X = sm.add_constant(data=X)   # add a constant to the model
model_mr = sm.OLS(endog=Y,exog=X)
results_mr = model_mr.fit()

In [24]:
results_mr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.514
Model:                            OLS   Adj. R-squared:                  0.514
Method:                 Least Squares   F-statistic:                     5708.
Date:                Mon, 25 Sep 2023   Prob (F-statistic):               0.00
Time:                        16:40:07   Log-Likelihood:            -2.9981e+05
No. Observations:               21613   AIC:                         5.996e+05
Df Residuals:                   21608   BIC:                         5.997e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -8.937e+05    1.3e+04    -68.898      0.000   -9.19e+05   -8.68e+05
bedrooms    1.855e+04   2186.988      8.484      0.000    1.43e+04    2.28e+04
bathrooms   5.401e+04   3304.360     16.346      0.000    4.75e+04    6.05e+04
view        1.164e+05   2349.627     49.556      0.000    1.12e+05    1.21e+05
grade       1.606e+05   2014.564     79.722      0.000    1.57e+05    1.65e+05
==============================================================================
Omnibus:                    18992.884   Durbin-Watson:                   1.964
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1893016.549
Skew:                           3.778   Prob(JB):                         0.00
Kurtosis:                      48.222   Cond. No.                         66.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

> The model is fitting the data quite well with an $R^2 = 0.531$. All the regression coefficients are statistically significant with their p-values being all closes to zero and, in any case, lower than the threshold significance level of $\alpha = 0.05$. There is no multicollinearity between the independent variables, being the Condition Number relatively low. 

If we want we can also try to improve the model, for example adding new predictor. Let's create a new feature: `is_renovated`, that is $1$ if the house has been renovated and $0$ otherwise. 

In [25]:
is_renovated = []
for el in df["yr_renovated"]:
    if el > 0:
      is_renovated.append(1)  
    else: 
      is_renovated.append(0) 

df['is_renovated'] = is_renovated

Now we can see if adding the new feature to the model it improved or not

In [26]:
Y = df['price']
X = df[['bedrooms','bathrooms','view','grade','is_renovated']]
X = sm.add_constant(data=X)   # add a constant to the model
model_mr = sm.OLS(endog=Y,exog=X)
results_mr = model_mr.fit()
results_mr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.521
Model:                            OLS   Adj. R-squared:                  0.521
Method:                 Least Squares   F-statistic:                     4709.
Date:                Mon, 25 Sep 2023   Prob (F-statistic):               0.00
Time:                        16:40:07   Log-Likelihood:            -2.9964e+05
No. Observations:               21613   AIC:                         5.993e+05
Df Residuals:                   21607   BIC:                         5.993e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -9.061e+05   1.29e+04    -70.318      0.000   -9.31e+05   -8.81e+05
bedrooms      1.878e+04   2169.733      8.655      0.000    1.45e+04     2.3e+04
bathrooms     5.095e+04   3282.351     15.524      0.000    4.45e+04    5.74e+04
view           1.12e+05   2343.360     47.783      0.000    1.07e+05    1.17e+05
grade         1.622e+05   2000.519     81.089      0.000    1.58e+05    1.66e+05
is_renovated   1.61e+05   8644.471     18.621      0.000    1.44e+05    1.78e+05
==============================================================================
Omnibus:                    18917.249   Durbin-Watson:                   1.968
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1839698.818
Skew:                           3.761   Prob(JB):                         0.00
Kurtosis:                      47.568   Cond. No.                         66.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [27]:
df[['bedrooms','bathrooms','view','grade','is_renovated']].corr()

,bedrooms,bathrooms,view,grade,is_renovated
bedrooms,1.000000,0.515884,0.079532,0.356967,0.018553
bathrooms,0.515884,1.000000,0.187737,0.664983,0.050260
view,0.079532,0.187737,1.000000,0.251321,0.104062
grade,0.356967,0.664983,0.251321,1.000000,0.014008
is_renovated,0.018553,0.050260,0.104062,0.014008,1.000000


Let's say you own a house in King County, WA that has 4 bedrooms, 2 bathrooms and it has no particuar view. Its construction grade is 8 and it has been recently renovated. We can se how much could you sell it for using the model.

In [28]:
# Define the features of your house
bedrooms = 4
bathrooms = 2
view = 0
grade = 8
is_renovated = 1

# Add a constant to the features
features = np.array([1, bedrooms, bathrooms, view, grade, is_renovated])

# Use the last model to predict the price
price = results_mr.predict(exog=features)

print("The estimated selling price of your house is:", round(price[0], 2), "dollars.")

The estimated selling price of your house is: 729642.01 dollars.


Using the model we definesd we can split the dataset into two training and test subsets, where 33% of the avaialable data should be used for testing. This way we can calculate the MAE for the model and see if the model overfit the data or if we have to improve the model. 

In [29]:
# Split the data into training and test subsets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

# Add a constant to the features of both subsets
X_train = sm.add_constant(data=X_train)
X_test = sm.add_constant(data=X_test)

# Fit the model to the training data
model = sm.OLS(endog=y_train, exog=X_train)
results = model.fit()

# Calculate the MAE for the training model
y_train_pred = results.predict(X_train)
mae_train = mean_absolute_error(y_train, y_train_pred)

# Use the testing data to predict the response variable using the fitted model
y_test_pred = results.predict(X_test)

# Calculate the MAE for the test model
mae_test = mean_absolute_error(y_test, y_test_pred)

print("The MAE of the training model is:", round(mae_train, 2))
print("The MAE of the test model is:", round(mae_test, 2))


The MAE of the training model is: 162253.46
The MAE of the test model is: 168075.5


We can see that the model is not overfitting the data, since the Training and Testing errors are not too far off from each other. We can still improve the model including more features

In [30]:
y = df['price']
X = df[['bedrooms', 'bathrooms', 'floors', 'waterfront', 'view', 'condition', 'grade', 'is_renovated']]
X = sm.add_constant(data=X)   # add a constant to the model

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

model = sm.OLS(endog=y_train,exog=X_train)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.553
Model:                            OLS   Adj. R-squared:                  0.553
Method:                 Least Squares   F-statistic:                     2237.
Date:                Mon, 25 Sep 2023   Prob (F-statistic):               0.00
Time:                        16:40:07   Log-Likelihood:            -1.9985e+05
No. Observations:               14480   AIC:                         3.997e+05
Df Residuals:                   14471   BIC:                         3.998e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.115e+06   1.95e+04    -57.105      0.000   -1.15e+06   -1.08e+06
bedrooms      1.271e+04   2504.575      5.076      0.000    7803.411    1.76e+04
bathrooms     6.796e+04   4013.371     16.933      0.000    6.01e+04    7.58e+04
floors       -3.605e+04   4490.785     -8.028      0.000   -4.49e+04   -2.72e+04
waterfront    5.231e+05    2.6e+04     20.095      0.000    4.72e+05    5.74e+05
view           7.86e+04   2953.251     26.616      0.000    7.28e+04    8.44e+04
condition     5.952e+04   3196.360     18.621      0.000    5.33e+04    6.58e+04
grade         1.684e+05   2360.437     71.332      0.000    1.64e+05    1.73e+05
is_renovated  1.666e+05   1.01e+04     16.541      0.000    1.47e+05    1.86e+05
==============================================================================
Omnibus:                    12000.224   Durbin-Watson:                   1.997
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1142654.763
Skew:                           3.428   Prob(JB):                         0.00
Kurtosis:                      45.976   Cond. No.                         125.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [31]:
y_train_pred = results.predict(X_train)
mae_train = mean_absolute_error(y_train, y_train_pred)
y_test_pred = results.predict(X_test)
mae_test = mean_absolute_error(y_test, y_test_pred)

print("The MAE of the training model is:", round(mae_train, 2))
print("The MAE of the test model is:", round(mae_test, 2))

The MAE of the training model is: 157981.36
The MAE of the test model is: 163517.71
